In [ ]:
!pip install -r requirements.txt

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import MSE
from neuralforecast.models import TimeXer, PatchTST, LSTM, NBEATS
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import torch
import pandas as pd

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

In [ ]:
df = pd.read_csv('./bitcoin.csv')
display(df.head())

In [ ]:
df = df[['time', 'open', 'Global liquidity']]

In [ ]:
df['time'] = pd.to_datetime(df['time'], unit='s')
display(df.head())

In [ ]:
df.sort_values(by='time', inplace=True)
display(df.head())

In [ ]:
df_filtered = df[df['time'] >= '2020-01-01']
display(df_filtered.head())

In [ ]:
df_filtered.shape

In [ ]:
df_filtered.tail()

In [ ]:
import numpy as np
df_trans = df_filtered.copy()

df_trans = df_trans.reset_index()

df_trans['y'] = df_trans['open']
df_trans['global'] = df_trans['Global liquidity']

lags = [1, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98]
for lag in lags:
    df_trans[f'global_lag_{lag}'] = df_trans['global'].shift(-lag)


df_final = df_trans.dropna().copy()


if 'time' in df_final.columns:
    df_final = df_final.rename(columns={'time': 'ds'})
elif 'Date' in df_final.columns:
    df_final = df_final.rename(columns={'Date': 'ds'})

exog_cols = ['global'] + [f'global_lag_{lag}' for lag in lags]

df_final['unique_id'] = 'BTC'
df_final = df_final[['ds', 'unique_id', 'y'] + exog_cols]

print("Success! Dataframe head:")
display(df_final.head())
df_filtered = df_final.copy()

In [ ]:
def run_btc_forecast_experiment(df_filtered, input_size=256, h=49, test_size=120):
    exog_list = [
        "global", "global_lag_1", "global_lag_7", "global_lag_14", "global_lag_21",
        "global_lag_28", "global_lag_35", "global_lag_42", "global_lag_49",
        "global_lag_56", "global_lag_63", "global_lag_70", "global_lag_77",
        "global_lag_84", "global_lag_91", "global_lag_98"
    ]

    timexer_model = TimeXer(
        h=h, input_size=input_size, n_series=1,
        patch_len=96, hidden_size=256, n_heads=8,
        e_layers=16, d_ff=512, factor=1, step_size=8,
        dropout=0.25, use_norm=True, scaler_type='standard',
        loss=MSE(), valid_loss=MSE(),
        early_stop_patience_steps=3, learning_rate=0.0001,
        batch_size=32*4
    )

    timexer_model_exog = TimeXer(
        h=h, input_size=input_size, n_series=1,
        futr_exog_list=exog_list,
        patch_len=96, hidden_size=256, n_heads=8,
        e_layers=16, d_ff=512, factor=1, step_size=8,
        dropout=0.3, use_norm=True, scaler_type='standard',
        loss=MSE(), valid_loss=MSE(),
        early_stop_patience_steps=3, learning_rate=0.0001,
        batch_size=32*4
    )

    patchtst_model = PatchTST(
        h=h, input_size=input_size,
        encoder_layers=6, n_heads=8, hidden_size=128,
        linear_hidden_size=512, dropout=0.20,
        fc_dropout=0.25, head_dropout=0.20,
        attn_dropout=0.3, patch_len=96, stride=8,
        revin=True, scaler_type='standard',
        loss=MSE(), valid_loss=MSE(),
        early_stop_patience_steps=3,
        learning_rate=0.0001, batch_size=32*4
    )

    lstm_model = LSTM(
        h=h, input_size=input_size, encoder_n_layers=3,
        encoder_hidden_size=256, encoder_bias=True,
        encoder_dropout=0.15, decoder_hidden_size=128,
        decoder_layers=2, loss=MSE(), valid_loss=MSE(),
        scaler_type='standard', early_stop_patience_steps=3,
        learning_rate=0.0001, batch_size=32*4
    )

    nbeats = NBEATS(
        h=h,
        input_size=256,
        stack_types=['identity', 'trend', 'seasonality'],
        n_blocks=[1, 1, 1],
        mlp_units=[[512, 512], [512, 512], [512, 512]],
        loss=MSE(),
        valid_loss=MSE(),
        learning_rate=1e-4,
        batch_size=128,
        early_stop_patience_steps=3,
        scaler_type='standard',
    )

    models = [
        timexer_model, timexer_model_exog,
        patchtst_model, lstm_model, nbeats
    ]

    nf = NeuralForecast(models=models, freq='D')

    train_df = df_filtered.iloc[:-test_size]
    nf.fit(df=train_df, val_size=test_size)

    all_predictions = []
    current_train_df = train_df.copy()

    num_iterations = test_size // h
    print(f"Running {num_iterations} rolling forecast steps...")

    for i in range(num_iterations):

        start_idx = len(train_df) + i * h
        end_idx = start_idx + h

        # Prepare exogenous slice
        exog_cols = ['unique_id', 'ds'] + exog_list
        current_futr_df = df_filtered.iloc[start_idx:end_idx][exog_cols]

        # NF predictions
        nf_forecast = nf.predict(df=current_train_df, futr_df=current_futr_df)

        all_predictions.append(nf_forecast)

        # Expand training set
        actual_slice = df_filtered.iloc[start_idx:end_idx]
        current_train_df = pd.concat([current_train_df, actual_slice], ignore_index=True)

    predictions_df = pd.concat(all_predictions, ignore_index=True)
    future_df = df_filtered.iloc[len(train_df):len(train_df)+test_size]

    rows_to_drop = future_df.shape[0] - predictions_df.shape[0]
    if rows_to_drop > 0:
        future_df = future_df.iloc[:-rows_to_drop]

    metrics = {
        "TimeXer": mean_squared_error(future_df['y'], predictions_df['TimeXer']),
        "TimeXer_Exog": mean_squared_error(future_df['y'], predictions_df['TimeXer1']),
        "PatchTST": mean_squared_error(future_df['y'], predictions_df['PatchTST']),
        "LSTM": mean_squared_error(future_df['y'], predictions_df['LSTM']),
        "NBEATS": mean_squared_error(future_df['y'], predictions_df['NBEATS']),
    }

    sorted_metrics = sorted(metrics.items(), key=lambda x: x[1])

    print("\n--- MSE Metrics (Lower is better) ---")
    for name, mse in sorted_metrics:
        print(f"{name}: {mse:.4f}")

    plt.figure(figsize=(16, 8))
    plt.plot(future_df['ds'], future_df['y'], label="Actual", color='black', linewidth=2)

    for col in ["TimeXer", "TimeXer1", "PatchTST", "LSTM", "NBEATS"]:
        plt.plot(future_df['ds'], predictions_df[col], label=col, linestyle='--')

    plt.title("Model Comparison")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.grid(True)
    plt.legend()
    plt.show()

    return predictions_df, metrics, sorted_metrics

In [ ]:
preds, metrics, sorted_metrics = run_btc_forecast_experiment(
    df_filtered,
    input_size=256,
    h=7,
    test_size=120
)

In [ ]:
preds, metrics, sorted_metrics = run_btc_forecast_experiment(
    df_filtered,
    input_size=256,
    h=14,
    test_size=120
)

In [ ]:
preds, metrics, sorted_metrics = run_btc_forecast_experiment(
    df_filtered,
    input_size=256,
    h=21,
    test_size=120
)

In [ ]:
preds, metrics, sorted_metrics = run_btc_forecast_experiment(
    df_filtered,
    input_size=256,
    h=28,
    test_size=120
)

In [ ]:
preds, metrics, sorted_metrics = run_btc_forecast_experiment(
    df_filtered,
    input_size=256,
    h=35,
    test_size=120
)

In [ ]:
preds, metrics, sorted_metrics = run_btc_forecast_experiment(
    df_filtered,
    input_size=256,
    h=42,
    test_size=120
)

In [ ]:
preds, metrics, sorted_metrics = run_btc_forecast_experiment(
    df_filtered,
    input_size=256,
    h=49,
    test_size=120
)

In [ ]:
preds, metrics, sorted_metrics = run_btc_forecast_experiment(
    df_filtered,
    input_size=256,
    h=56,
    test_size=120
)

In [ ]:
preds, metrics, sorted_metrics = run_btc_forecast_experiment(
    df_filtered,
    input_size=256,
    h=63,
    test_size=120
)

In [ ]:
preds, metrics, sorted_metrics = run_btc_forecast_experiment(
    df_filtered,
    input_size=256,
    h=70,
    test_size=120
)